In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import time
import datetime

from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

import os
from tqdm import tqdm_notebook

# 번개장터 이미지크롤링

### 주요상품
- 310	여성의류	w_wear
- 320	남성의류	m_wear
- 400	패션액세서리	acc
- 405	신발	shoes
- 420	시계_쥬얼리	jewelry
- 430	가방	bag
- 600	디지털_가전	electronic
- 700	스포츠_레저	sports
- 750	차량_오토바이	car


### 비주류상품
- 410	뷰티	beauty
- 500	유아	baby
- 800	생활_가공식품	living
- 810	가구_인테리어	interior
- 900	도서_티켓_문구	book_stationery
- 910	스타굿즈	stargoods
- 920	음반_악기	music
- 930	키덜트	kidult
- 980	반려동물	pet
- 990	수집품	collect
- 999	기타	others

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [3]:
# 드라이버 실행
driver = webdriver.Chrome('/Users/ppangppang/Desktop/ssac/02.Python_/Python_Basic_Class/chromedriver')

In [149]:
# cat 별 링크 들어가게 세팅
cat_list = [310,320,400,405,420,430,600,700,750]
cat_name = ['w_wear','m_wear','acc','shoes','jewelry','bag','electronic','sports','car']
pages = [1,2,3]
# 카테고리별 링크 담기
total_df = []

for cat in cat_list:
    # category별 dataframe 생성
    cat_df=pd.DataFrame()
    link=[]    
    for page in pages:
        url = f'https://m.bunjang.co.kr/categories/{cat}?page={page}&req_ref=popular_category'
        driver.get(url)
        time.sleep(2)
    
        # 페이지마다 사진 100장씩
        for i in range(1,101):
            try:
                link.append(driver.find_element_by_xpath(f'//*[@id="root"]/div/div/div[4]/div/div[4]/div/div[{i}]/a/div[1]/img').get_attribute('src'))
            except: pass
        print(f'{cat_name[cat_list.index(cat)]}항목 {page}페이지 완료')
        
    cat_df['link'] = link
    cat_df['category'] = cat_name[cat_list.index(cat)]
    total_df.append(cat_df)
    print(f'{cat_name[cat_list.index(cat)]}항목 작업 완료\n')
    
print('전체작업 완료')

310항목 1페이지 완료
310항목 2페이지 완료
310항목 3페이지 완료
w_wear항목 작업 완료

320항목 1페이지 완료
320항목 2페이지 완료
320항목 3페이지 완료
m_wear항목 작업 완료

400항목 1페이지 완료
400항목 2페이지 완료
400항목 3페이지 완료
acc항목 작업 완료

405항목 1페이지 완료
405항목 2페이지 완료
405항목 3페이지 완료
shoes항목 작업 완료

410항목 1페이지 완료
410항목 2페이지 완료
410항목 3페이지 완료
beauty항목 작업 완료

420항목 1페이지 완료
420항목 2페이지 완료
420항목 3페이지 완료
jewelry항목 작업 완료

430항목 1페이지 완료
430항목 2페이지 완료
430항목 3페이지 완료
bag항목 작업 완료

600항목 1페이지 완료
600항목 2페이지 완료
600항목 3페이지 완료
electronic항목 작업 완료

700항목 1페이지 완료
700항목 2페이지 완료
700항목 3페이지 완료
sports항목 작업 완료

750항목 1페이지 완료
750항목 2페이지 완료
750항목 3페이지 완료
car항목 작업 완료

전체작업 완료


In [152]:
# 각 작업 파일 저장
for i in range(len(cat_list)):
    total_df[0].to_csv(f'/Users/ppangppang/Desktop/ssac/Final_project/2.Processed/image/image_df/{cat_name[0]}.csv',index=False)
    total_df[i].to_csv(f'/Users/ppangppang/Desktop/ssac/Final_project/2.Processed/image/image_df/{cat_name[i]}.csv',index=False)

# 번개장터 크롤링 이미지 저장

In [ ]:
# 파일 불러오기
cat_name = ['w_wear','m_wear','acc','shoes','jewelry','bag','electronic','sports','car']
total_df=[]
for cat in cat_name:
    temp = pd.read_csv(f'/Users/ppangppang/Desktop/ssac/Final_project/2.Processed/image/image_df/{cat_name[0]}.csv')
    total_df.append(temp)

In [ ]:
from PIL import Image

for cat in range(len(total_df)):
    for i in tqdm_notebook(range(len(total_df[cat])),desc="Image download"):
        url = total_df[0]['link'].iloc[i]
        os.system("curl " + url + f" > /Users/ppangppang/Desktop/ssac/Final_project/2.Processed/image/{cat_name[cat]}/c_{i}.jpg")

# 카테고리 재분류 크롤링

In [28]:
# 드라이버 실행
driver = webdriver.Chrome('/Users/ppangppang/Desktop/ssac/02.Python_/Python_Basic_Class/chromedriver')

In [29]:
# cat 별 링크 들어가게 세팅
cat_list = ['스킨로션','바디','향수','네일아트','다이어트','인테리어','침대','모자','목도리','아디다스운동화','나이키운동화','닌텐도','마사지기']

pages = list(range(1,101))
# pages = [1,2]
# 카테고리별 링크 담기
total_df = []

for cat in cat_list:
    # category별 dataframe 생성
    cat_df=pd.DataFrame()
    link=[]    
    for page in pages:
        url = f'https://m.bunjang.co.kr/search/products?q={cat}&order=score&page={page}'
        driver.get(url)
    
        # 페이지마다 사진 100장씩
        for i in range(1,101):
            try:
                link.append(driver.find_element_by_xpath(f'//*[@id="root"]/div/div/div[4]/div/div[4]/div/div[{i}]/a/div[1]/img').get_attribute('src'))
            except: pass
        print(f'{cat}항목 {page}페이지 완료')
        
    cat_df['link'] = link
    cat_df['category'] = cat
    total_df.append(cat_df)
    print(f'{cat}항목 작업 완료\n')
    
print('전체작업 완료')

스킨로션항목 1페이지 완료
스킨로션항목 2페이지 완료
스킨로션항목 3페이지 완료
스킨로션항목 4페이지 완료
스킨로션항목 5페이지 완료
스킨로션항목 6페이지 완료
스킨로션항목 7페이지 완료
스킨로션항목 8페이지 완료
스킨로션항목 9페이지 완료
스킨로션항목 10페이지 완료
스킨로션항목 11페이지 완료
스킨로션항목 12페이지 완료
스킨로션항목 13페이지 완료
스킨로션항목 14페이지 완료
스킨로션항목 15페이지 완료
스킨로션항목 16페이지 완료
스킨로션항목 17페이지 완료
스킨로션항목 18페이지 완료
스킨로션항목 19페이지 완료
스킨로션항목 20페이지 완료
스킨로션항목 21페이지 완료
스킨로션항목 22페이지 완료
스킨로션항목 23페이지 완료
스킨로션항목 24페이지 완료
스킨로션항목 25페이지 완료
스킨로션항목 26페이지 완료
스킨로션항목 27페이지 완료
스킨로션항목 28페이지 완료
스킨로션항목 29페이지 완료
스킨로션항목 30페이지 완료
스킨로션항목 31페이지 완료
스킨로션항목 32페이지 완료
스킨로션항목 33페이지 완료
스킨로션항목 34페이지 완료
스킨로션항목 35페이지 완료
스킨로션항목 36페이지 완료
스킨로션항목 37페이지 완료
스킨로션항목 38페이지 완료
스킨로션항목 39페이지 완료
스킨로션항목 40페이지 완료
스킨로션항목 41페이지 완료
스킨로션항목 42페이지 완료
스킨로션항목 43페이지 완료
스킨로션항목 44페이지 완료
스킨로션항목 45페이지 완료
스킨로션항목 46페이지 완료
스킨로션항목 47페이지 완료
스킨로션항목 48페이지 완료
스킨로션항목 49페이지 완료
스킨로션항목 50페이지 완료
스킨로션항목 51페이지 완료
스킨로션항목 52페이지 완료
스킨로션항목 53페이지 완료
스킨로션항목 54페이지 완료
스킨로션항목 55페이지 완료
스킨로션항목 56페이지 완료
스킨로션항목 57페이지 완료
스킨로션항목 58페이지 완료
스킨로션항목 59페이지 완료
스킨로션항목 60페이지 완료
스킨로션항목 61페이지 완료
스킨로션항목 62페이지 완료
스킨로션항목 63페이지 완료
스

인테리어항목 37페이지 완료
인테리어항목 38페이지 완료
인테리어항목 39페이지 완료
인테리어항목 40페이지 완료
인테리어항목 41페이지 완료
인테리어항목 42페이지 완료
인테리어항목 43페이지 완료
인테리어항목 44페이지 완료
인테리어항목 45페이지 완료
인테리어항목 46페이지 완료
인테리어항목 47페이지 완료
인테리어항목 48페이지 완료
인테리어항목 49페이지 완료
인테리어항목 50페이지 완료
인테리어항목 51페이지 완료
인테리어항목 52페이지 완료
인테리어항목 53페이지 완료
인테리어항목 54페이지 완료
인테리어항목 55페이지 완료
인테리어항목 56페이지 완료
인테리어항목 57페이지 완료
인테리어항목 58페이지 완료
인테리어항목 59페이지 완료
인테리어항목 60페이지 완료
인테리어항목 61페이지 완료
인테리어항목 62페이지 완료
인테리어항목 63페이지 완료
인테리어항목 64페이지 완료
인테리어항목 65페이지 완료
인테리어항목 66페이지 완료
인테리어항목 67페이지 완료
인테리어항목 68페이지 완료
인테리어항목 69페이지 완료
인테리어항목 70페이지 완료
인테리어항목 71페이지 완료
인테리어항목 72페이지 완료
인테리어항목 73페이지 완료
인테리어항목 74페이지 완료
인테리어항목 75페이지 완료
인테리어항목 76페이지 완료
인테리어항목 77페이지 완료
인테리어항목 78페이지 완료
인테리어항목 79페이지 완료
인테리어항목 80페이지 완료
인테리어항목 81페이지 완료
인테리어항목 82페이지 완료
인테리어항목 83페이지 완료
인테리어항목 84페이지 완료
인테리어항목 85페이지 완료
인테리어항목 86페이지 완료
인테리어항목 87페이지 완료
인테리어항목 88페이지 완료
인테리어항목 89페이지 완료
인테리어항목 90페이지 완료
인테리어항목 91페이지 완료
인테리어항목 92페이지 완료
인테리어항목 93페이지 완료
인테리어항목 94페이지 완료
인테리어항목 95페이지 완료
인테리어항목 96페이지 완료
인테리어항목 97페이지 완료
인테리어항목 98페이지 완료
인테리어항목 9

TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.940
  (Session info: chrome=96.0.4664.110)


In [ ]:
# 각 작업 파일 저장
for i in range(len(cat_list)):
    total_df[0].to_csv(f'/Users/ppangppang/Desktop/ssac/Final_project/2.Processed/image/image_df/{cat_name[0]}.csv',index=False)    
    total_df[i].to_csv(f'/Users/ppangppang/Desktop/ssac/Final_project/2.Processed/image/image_df/{cat_list[i]}.csv',index=False)

In [ ]:
# 파일 불러오기
cat_list = ['스킨로션','바디','향수','네일아트','다이어트','인테리어','침대','모자','목도리','아디다스운동화','나이키운동화','닌텐도','마사지기']
total_df=[]
for cat in cat_list:
    temp = pd.read_csv(f'/Users/ppangppang/Desktop/ssac/Final_project/2.Processed/image/image_df/{cat}.csv')
    total_df.append(temp)

In [ ]:
from PIL import Image

# cat_list = ['스킨로션','바디','향수','네일아트','다이어트','인테리어','침대','모자','목도리','아디다스운동화','나이키운동화','닌텐도','마사지기']
cat_name = ['skin','body','perfume','nail','diet','interior','bed','hat','muffler','adidas','nike','nintendo','massage']
for cat in range(len(total_df)):
    for i in tqdm_notebook(range(len(total_df[cat])),desc="Image download"):
        url = total_df[cat]['link'].iloc[i]
        os.system("curl " + url + f" > /Users/ppangppang/Desktop/ssac/Final_project/2.Processed/image/{cat_list[cat]}/{cat_name[cat]}{i}.jpg")